In [ ]:
import mcstasscript as ms
import make_powder_instrument
from mcstasutils import plot
import quizlib

In [ ]:
quiz = quizlib.Powder_Quiz()

# Powder diffraction exercise

In this notebook you will work with a McStas model of a simplified powder diffraction instrument.
You will have to answer questions in the notebook by working with this model, both by running simulations and expanding the model.
We will use the Python McStas API McStasScript to work with the instrument, you can find documentation [here](https://mads-bertelsen.github.io).

## Get the instrument object
First we need the McStas instrument object. Here it is retrieved from a local python function that generates it.

In [ ]:
instrument = make_powder_instrument.make()

## Investigate instrument
The first task is to investigate the instrument object `instrument` using some of the available methods available on that object. Each method that show something about the instrument starts with the word show, so you can use tab to autocomplete in the cell to see the relevant methods.

In particular, look at what parameters are available and take a look at the instrument geometry.

In [ ]:
instrument = make_powder_instrument.make()
instrument.show_parameters()

In [ ]:
# instrument.show_instrument(format='window')
instrument.show_diagram()

### Set sample size
Select appropriate sample size given Vanadium has a macroscopic scattering cross section $\Sigma$ of around 0.35 cm^-1. A neutron beam with intensity $I_0$ that travel in a media a distance of $z$ will be attenuated as the neutrons scatter in the material, and the remaining intensity $I$ can be calculated with the Beer-Lambert law:

$$
  I = I_0 e^{-z\Sigma}
$$

For our experiment we want to observe neutrons that scattered once, as neutrons that scatter more than once would be considered background.

What sample depth would be appropriate?

- A: 10 cm
- B: 1 cm
- C: 1 mm

In [ ]:
# answer
import numpy as np
fraction_left = np.exp(-0.35*1)
print("scattering probability", 1 - fraction_left)
print("single scattering probability", (1 - fraction_left)*fraction_left)
print("double scattering probability", (1 - fraction_left)**2*fraction_left)
print("single to multiple scattering probability ", (1 - fraction_left)*fraction_left/(1-fraction_left))

In [ ]:
quiz.question_1("B")

### Set sample size in simulated instrument
Use the *set_parameters* method to set the sample thickness, here using the parameter *sample_radius*. Set the wavelength range using "l_min" and "l_max" to 2.5 - 2.501.

In [ ]:
wavelength = 2.5
instrument.set_parameters(sample_radius=0.005, l_min=wavelength, l_max=wavelength + 0.001)

In [ ]:
quiz.question_2(instrument)

### Run simulation
Run the simulation in this configuration and plot the data.

In [ ]:
instrument.settings(ncount=4.0e6, mpi=4, suppress_output=True, NeXus=True)

In [ ]:
instrument.settings(mpi=2)

In [ ]:
data = instrument.backengine()

In [ ]:
%matplotlib widget

ms.make_sub_plot(data)

### Time resolution
Whats the relative uncertainty on the time observed at the sample position, FWHM?

Insert value as a percentage.

In [ ]:
# answer
print("delta t/t", (1.0453E5 - 1.0173E5)/(0.5*(1.0453E5 + 1.0173E5))*100, "%")

In [ ]:
quiz.question_3(2.715)

This will correspond to the wavelength resolution at this wavelength, generally we need less than 1% for powder diffraction, is this sufficient?

- A: yes
- B: no

In [ ]:
quiz.question_4("B")

How could we improve this?
- A: monochromator
- B: chopper
- C: velocity selector
- D: shorter guide

In [ ]:
quiz.question_5("B")

The chopper needs to be inserted outside of the ESS monolith which has a radius of 6 m. Where in the McStas component sequence would that have to go?

- A: before feeder
- B: after feeder
- C: before expanding
- D: after expanding

In [ ]:
quiz.question_6("B")

Add the chopper with the following parameters:
- `yheight`: 0.05 m
- `radius`: 0.35 m
- `nslit`: 1.0
- `theta_0`: 7.0 deg
- `delay`: To the variable calculated in the instrument (use quotation marks)
- `nu`: A calculation using the `frequency_multiplier` variable, "frequency_multiplier*14.0"


In [ ]:
# simple answer, to be used to skip this question (also sets position, next question)
#make_powder_instrument.add_chopper(instrument)

In [ ]:
# Answer: Adding the chopper manually
make_powder_instrument.add_chopper_calculations(instrument)
    
chopper = instrument.add_component("chopper", "DiskChopper", after="feeder")
chopper.theta_0 = 7.0
chopper.nslit = 1
chopper.radius = 0.35
chopper.yheight = 0.05
chopper.nu = "frequency_multiplier*14.0"
chopper.delay = "delay"  # Variable with calculated delay

In [ ]:
quiz.question_7(instrument)

Set the chopper position, a variable is already created with the name "chopper_position" and value of 6.5 meters, this is the distance relative to the neutron Source.

In [ ]:
chopper = instrument.get_component("chopper")
chopper.set_AT("chopper_position", RELATIVE="Source")

In [ ]:
quiz.question_8(instrument)

Run the simulation with the parameter frequency_multiplier set to 3 and plot the data

In [ ]:
# Answer
instrument.set_parameters(frequency_multiplier=3)
data = instrument.backengine()

In [ ]:
# Answer
ms.make_sub_plot(data)

Whats the relative time uncertainty with this setup? Insert the answer as a percentage.

In [ ]:
print("delta t/t", (1.0315E5 - 1.0267E5)/1.0289E5*100, "%")

In [ ]:
quiz.question_9(0.4665)

### Set parameters for run with Si sample
Set wavelength interval from 0.5 to 4.0 Å
Select sample called "sample_Si", you will need to use '"string"'. The detector_height also needs to be set to 1.5 m.

In [ ]:
instrument.set_parameters(sample_choice='"sample_Si"', l_min=0.5, l_max=4.0, detector_height=1.5)

In [ ]:
quiz.question_10(instrument)

Now we are ready to run the simulation

In [ ]:
data = instrument.backengine()

The event data can be visualized with the supplied *plot* function.

In [ ]:
plot(data)

### Describe the data
What do we see in the plots from event data?

- A: Inelastic peaks
- B: Magnetic scattering
- C: Bragg peaks

In [ ]:
quiz.question_11("C")

Set the instrument settings to a reasonable value for the number of neutrons and MPI processes. This will be used in the following simulations.

In [ ]:
instrument.settings(ncount=4.0e8, mpi=4, suppress_output=True, NeXus=True)

In [ ]:
instrument.settings(mpi=2)

## Run reference sample: Si

In [ ]:
# Set frequency multiplier to 3 for high resolution
instrument.set_parameters(sample_choice='"sample_Si"', frequency_multiplier=3, detector_height=1.5)
instrument.settings(output_path="output_sample_Si")

In [ ]:
data_si = instrument.backengine()

In [ ]:
plot(data_si, orders_of_mag=5)

## Run calibration sample: Vanadium

In [ ]:
instrument.set_parameters(sample_choice='"sample_Vanadium"')
instrument.settings(output_path="output_sample_vanadium")

data_vanadium = instrument.backengine()

In [ ]:
plot(data_vanadium, orders_of_mag=5)

## Run main sample: LBCO

In [ ]:
instrument.set_parameters(sample_choice='"sample_LBCO"')
instrument.settings(output_path="output_sample_LBCO")

data_sample_lbco = instrument.backengine()

In [ ]:
plot(data_sample_lbco, orders_of_mag=5)